In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dfData = pd.read_csv('events_up_to_01062018.csv')
dfTrain = pd.read_csv('labels_training_set.csv')
dfTest = pd.read_csv('trocafone_kaggle_test.csv')
dfSample = pd.read_csv('trocafone_kaggle_submit_sample_all_0.csv')

c:\users\ignacio\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Crea un df con todos los usuarios
dfTest['label'] = 0
dfUsers = pd.concat([dfTrain,dfTest])
dfTest = dfTest.drop('label', axis=1)

In [4]:
#Le asigna a cada usuario su cantidad de eventos
def pesoTiempo(time):
    mes = int(time[6])
    dia = int(time[8:10])
    if mes < 5:
        return 1
    elif mes == 5 and dia > 15:
        return 20
    else:
        return 10
    
dfPE = dfData[['person', 'event', 'timestamp']]
dfPE['puntajeTiempo'] = dfPE['timestamp'].apply(pesoTiempo)
dfPE = pd.get_dummies(dfPE, columns=['event'])
columnasEventos = dfPE.columns.values[3:]
for eventoColumna in columnasEventos:
    dfPE[eventoColumna] *= dfPE['puntajeTiempo']
dfPE = dfPE.drop(['timestamp', 'puntajeTiempo'], axis=1).groupby('person').sum()
dfUsers = dfUsers.sort_values('person').merge(dfPE, on='person')

c:\users\ignacio\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [5]:
#Casteo dfUsers a int32
personas = dfUsers['person']
dfUsers.drop('person', axis=1, inplace=True)
dfUsers = dfUsers.astype('int32')
dfUsers['person'] = personas

In [6]:
#Agrego columna con ultima visita al sitio
temp = dfData.groupby('person')['timestamp'].max().to_frame()
dfUsers = dfUsers.merge(temp, on='person')
dfUsers['month'] = dfUsers['timestamp'].str[5:7]
dfUsers['day'] = dfUsers['timestamp'].str[8:10]
dfUsers[['day','month']] = dfUsers[['day','month']].apply(pd.to_numeric)
dfUsers['last_visit'] = dfUsers['month']*30 + dfUsers['day']
dfUsers = dfUsers.drop(['timestamp','month','day'], axis=1)

In [7]:
#Agrego columna con ultima compra
temp = dfData.loc[dfData['event'] == 'conversion'].groupby('person')['timestamp'].max().to_frame()
dfUsers = dfUsers.merge(temp, how='left', on='person')
dfUsers = dfUsers.fillna('000000000000000000')
dfUsers['month'] = dfUsers['timestamp'].str[5:7]
dfUsers['day'] = dfUsers['timestamp'].str[8:10]
dfUsers[['day','month']] = dfUsers[['day','month']].apply(pd.to_numeric)
dfUsers['last_conversion'] = dfUsers['month']*30 + dfUsers['day']
dfUsers = dfUsers.drop(['timestamp','month','day'], axis=1)

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

df_y = dfTrain['label']
df_X = dfTrain.drop('label', axis=1).merge(dfUsers, on='person').drop(['person','label'], axis=1)

GBC = GradientBoostingClassifier(random_state=23)
scores = cross_val_score(GBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()

0.8604562075072076

In [9]:
GBC2 = GradientBoostingClassifier(random_state=23)
GBC2.fit(df_X, df_y)
df_p = dfTest.merge(dfUsers, on='person').drop(['person','label'], axis=1)
temp = GBC2.predict_proba(df_p)
submit = []
for l in temp:
    submit.append(l[1])
dfTest['label'] = submit

In [10]:
dfTest.to_csv('trocafone_kaggle_submit.csv', index=False)

In [11]:
pd.read_csv('trocafone_kaggle_submit.csv')

,person,label
0,4886f805,0.007274
1,0297fc1e,0.019628
2,2d681dd8,0.010933
3,cccea85e,0.064468
4,4c8a8b93,0.023476
5,29ebb414,0.010072
6,3dc1950f,0.085586
7,8ea4c165,0.021526
8,d8cfe234,0.032824
9,d6bc64df,0.019214
